In [1]:
import plotly.express as px
import pandas as pd

In [2]:
umsatz = pd.read_csv('../Data/processed_umsatz.csv')
gewerbe = pd.read_csv('../Data/Umsatzdaten/Gastgewerbe.csv')
umsatz.columns

Index(['Unnamed: 0', 'Gastgewerbe', 'Umsatz', 'Datum'], dtype='object')

In [3]:
umsatz["Datum"] = pd.to_datetime(umsatz["Datum"], format="%Y-%m-%d")
umsatz["Monat"] = umsatz["Datum"].dt.month
umsatz["Jahr"] = umsatz["Datum"].dt.year
umsatz = umsatz.dropna(subset=['Umsatz'])
umsatz.drop(columns=['Unnamed: 0'], inplace=True)
umsatz.head(3)

,Gastgewerbe,Umsatz,Datum,Monat,Jahr
0,WZ08-55,92.3,1994-01-01,1,1994
1,WZ08-551,91.9,1994-01-01,1,1994
2,WZ08-552,97.3,1994-01-01,1,1994


In [4]:
merged_df = pd.merge(umsatz, gewerbe, left_on='Gastgewerbe', right_on='GastgewerbeCode')
merged_df.head(3)

,Gastgewerbe,Umsatz,Datum,Monat,Jahr,GastgewerbeCode,GastgewerbeBezeichner
0,WZ08-55,92.3,1994-01-01,1,1994,WZ08-55,Beherbergung
1,WZ08-551,91.9,1994-01-01,1,1994,WZ08-551,"Hotels, Gasthöfe und Pensionen"
2,WZ08-552,97.3,1994-01-01,1,1994,WZ08-552,Ferienunterkünfte u. ähnliche Beherbergungsstä...


In [5]:
avg_umsatz_jahr =  merged_df.groupby(['Monat', 'Gastgewerbe',  'GastgewerbeBezeichner'])['Umsatz'].mean().reset_index()

fig = px.line(avg_umsatz_jahr, x='Monat', y='Umsatz', color='GastgewerbeBezeichner', title='Durchschnittliche Umsätze pro Jahr und Gastgewerbe')
fig.show()

c:\Users\Ready2Go\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1985: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


In [6]:
avg_umsatz = avg_umsatz_jahr.groupby('Gastgewerbe')['Umsatz'].mean().reset_index()
avg_umsatz = avg_umsatz.rename(columns={'Umsatz': 'UmsatzAvg'})
avg_umsatz_jahr = pd.merge(avg_umsatz_jahr, avg_umsatz, left_on='Gastgewerbe', right_on='Gastgewerbe')
avg_umsatz_jahr.head(3)

,Monat,Gastgewerbe,GastgewerbeBezeichner,Umsatz,UmsatzAvg
0,1,WZ08-55,Beherbergung,78.083871,102.754211
1,1,WZ08-55-01,Gastgewerbe,92.448387,111.274310
2,1,WZ08-551,"Hotels, Gasthöfe und Pensionen",78.003226,102.384435


In [7]:
avg_umsatz_jahr['Differenz'] = avg_umsatz_jahr['Umsatz'] - avg_umsatz_jahr['UmsatzAvg']
df_grouped = avg_umsatz_jahr.groupby('Monat')['Differenz'].sum().reset_index()

fig = px.bar(df_grouped, x='Monat', y='Differenz', 
             title='Summierte Differenz zwischen Umsatz und UmsatzAvg pro Monat',
             labels={'Monat': 'Monat', 'Differenz': 'Summierte Differenz (Umsatz - UmsatzAvg)'})

fig.show()

Unsere Empfehlung für eine neu Erüffnung eines Gastrobetriebs wäre im Januar oder Feburar. Hier sind die Umsätze im Schnitt am Niedrigsten, woraus man einen generell, geringeren Andrang an Gästen erschließen. Wenn hier eine Neueröffnung statt finden würde, hätte man noch genug Zeit um sich auf die Hauptsaison vorzu bereiten. Hier denken wir an das Einarbeiten von Mitarbeitern und das erkennen von unerwarteten Problemen. 